爬取豆瓣top25的电影及其详细信息

In [152]:
import requests
from bs4 import BeautifulSoup
import traceback
import re
import time
from lxml import etree

爬取函数

In [138]:
def getHTMLtext(url):
    try:
        headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36','Cookie': 'bid=b76oHIKgOYQ; __yadk_uid=A5CvFGluucGcgKRvopNnTiUhygDYF1A6; __gads=ID=12bd23effde3ba34-22eb143a6bc6007a:T=1615786185:RT=1615786185:S=ALNI_MaK5tliCmx5Y0xMJ10ztIvB7Fzi_g; __utmc=30149280; __utmc=223695111; ll="108288"; _vwo_uuid_v2=DDB059D61CEF6AC3F8E830D8EB85CC544|ef4bb02815e4c0cd1fea9c5751fa261c; ap_v=0,6.0; dbcl2="208638353:DHfDDrZcnAk"; ck=-E9w; __utma=30149280.1855460508.1615786182.1621682045.1621685240.5; __utmb=30149280.0.10.1621685240; __utmz=30149280.1621685240.5.3.utmcsr=accounts.douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/; __utma=223695111.1041649550.1615786182.1621682045.1621685240.5; __utmb=223695111.0.10.1621685240; __utmz=223695111.1621685240.5.3.utmcsr=accounts.douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/; _pk_ref.100001.4cf6=%5B%22%22%2C%22%22%2C1621685240%2C%22https%3A%2F%2Faccounts.douban.com%2F%22%5D; _pk_id.100001.4cf6=7f2117d434703f1a.1615786181.5.1621685240.1621682048.; _pk_ses.100001.4cf6=*'}
        r=requests.get(url,headers=headers,timeout=50)
        r.raise_for_status()
        r.encoding=r.apparent_encoding
        return r.content
    except:
        return ""

爬取页面中的详细页面的url

In [144]:
def getdetailurl(list,html):
    try:
        html=etree.HTML(html)
        detailurl=html.xpath("//div[@class='hd']/a/@href")
        for i in range(len(detailurl)):
            Info={}
            Info["detailurl"]=detailurl[i]
            list.append(Info)
        return list
    except:
        return ""
        
        
    

在详细页面中寻找信息并存储

In [150]:
def  getInfolist(ilt,html):
    html=etree.HTML(html)
    detailname=html.xpath("/html/body/div[3]/div[1]/h1/span[1]/text()")
    director=html.xpath("/html/body/div[3]/div[1]/div[3]/div[1]/div[1]/div[1]/div[1]/div[2]/span[1]/span[2]/a/text()")
    detaildirector=''
    for i in range(len(director)):
        detaildirector=detaildirector+director[i]+'/'
    role=html.xpath("//span/a[@rel='v:starring']/text()")
    detailrole=''
    for i in range(len(role)):
        detailrole=detailrole+role[i]+'/'
    category=html.xpath("//span[@property='v:genre']/text()")
    detailcategory=''
    for i in range(len(category)):
        detailcategory=detailcategory+category[i]+'/'
    country=html.xpath("/html/body/div[3]/div[1]/div[3]/div[1]/div[1]/div[1]/div[1]/div[2]/text()")
    country=find_(country)
    detailcontry=country[0]
    time=html.xpath("//span[@property='v:initialReleaseDate']/text()")
    detailtime=''
    for i in range(len(time)):
        detailtime=detailtime+time[i]+'/'
    detaillength=html.xpath("//span[@property='v:runtime']/text()")
    detailevaluate=html.xpath("/html/body/div[3]/div[1]/div[3]/div[1]/div[1]/div[1]/div[2]/div[1]/div[2]/strong/text()")
    
    Info={}
    Info["name"]=detailname
    Info["director"]=detaildirector
    Info["role"]=detailrole
    Info["category"]=detailcategory
    Info["country"]=detailcontry
    Info["time"]=detailtime
    Info["length"]=detaillength
    Info["evaluate"]=detailevaluate
    ilt.append(Info)
    

发现列表中的中文，便于找到电影的国家名

In [146]:
def find_(mydata):
    new_data = []
    p = re.compile(r'[\u4e00-\u9fa5]')
    for i in range(len(mydata)):
        data = re.findall(p, mydata[i])
        result = ''.join(data)
        new_data.append(result)
    box = []
    for i in range(len(new_data)):
        if new_data[i] != '':
            box.append(new_data[i])
    return box

主函数

In [151]:
def main():
    start_url='https://movie.douban.com/top250?start='
    detailurlist=[]
    for i in range(1):
        generalurl=start_url+str(25*i)
        generalHTMl=getHTMLtext(generalurl)
        getdetailurl(detailurlist,generalHTMl)
        print('爬完第'+str(i+1)+'页的详情url')
    Infolist=[]
    for i in range(len(detailurlist)):
        detailurl=detailurlist[i]['detailurl']
        detailHTML=getHTMLtext(detailurl)
        getInfolist(Infolist,detailHTML)
    print(Infolist)
    
        
        
main()



爬完第1页的详情url
[{'name': ['肖申克的救赎 The Shawshank Redemption'], 'director': '弗兰克·德拉邦特/', 'role': '蒂姆·罗宾斯/摩根·弗里曼/鲍勃·冈顿/威廉姆·赛德勒/克兰西·布朗/吉尔·贝罗斯/马克·罗斯顿/詹姆斯·惠特摩/杰弗里·德曼/拉里·布兰登伯格/尼尔·吉恩托利/布赖恩·利比/大卫·普罗瓦尔/约瑟夫·劳格诺/祖德·塞克利拉/保罗·麦克兰尼/芮妮·布莱恩/阿方索·弗里曼/V·J·福斯特/弗兰克·梅德拉诺/马克·迈尔斯/尼尔·萨默斯/耐德·巴拉米/布赖恩·戴拉特/唐·麦克马纳斯/', 'category': '剧情/犯罪/', 'country': '美国', 'time': '1994-09-10(多伦多电影节)/1994-10-14(美国)/', 'length': ['142分钟'], 'evaluate': ['9.7']}, {'name': ['霸王别姬'], 'director': '陈凯歌/', 'role': '张国荣/张丰毅/巩俐/葛优/英达/蒋雯丽/吴大维/吕齐/雷汉/尹治/马明威/费振翔/智一桐/李春/赵海龙/李丹/童弟/沈慧芬/黄斐/徐杰/', 'category': '剧情/爱情/同性/', 'country': '中国大陆中国香港', 'time': '1993-07-26(中国大陆)/1993-01-01(中国香港)/', 'length': ['171分钟'], 'evaluate': ['9.6']}, {'name': ['阿甘正传 Forrest Gump'], 'director': '罗伯特·泽米吉斯/', 'role': '汤姆·汉克斯/罗宾·怀特/加里·西尼斯/麦凯尔泰·威廉逊/莎莉·菲尔德/海利·乔·奥斯蒙/迈克尔·康纳·亨弗里斯/哈罗德·G·赫瑟姆/山姆·安德森/伊俄涅·M·特雷奇/彼得·道博森/希芳·法隆/伊丽莎白·汉克斯/汉娜·豪尔/克里斯托弗·琼斯/罗布·兰德里/杰森·麦克奎尔/桑尼·施罗耶/艾德·戴维斯/丹尼尔C.斯瑞派克/大卫·布里斯宾/德博拉·麦克蒂尔/艾尔·哈林顿/阿非莫·奥米拉/约翰·沃德斯塔德/迈克尔·伯吉斯/埃里克·安德伍德/拜伦·明斯/斯蒂芬·布吉格沃特/约翰·威廉·高尔特/希拉里·沙普兰/伊莎贝尔·罗斯/理查德·